# Extract Contours of DAVIS 2016 Annotations

In this notebook the contour points of DAVIS 2016 annotations are saved as numpy arrays.

In [2]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import pandas as pd
import re
from scipy import spatial
from skimage import measure

%matplotlib inline

## Functions

In [3]:
def save_image_with_contour(image, contour, path):
    '''Saves image plotted with contour.'''
    
    fig, ax = plt.subplots()
    
    # Plot contour
    ax.scatter(contour[:, 0], contour[:, 1], s=1)
    
    # Plot image
    ax.imshow(image)
    ax.axis('image')
    ax.set_xticks([])
    ax.set_yticks([])
    
    # Save image
    plt.savefig(path, dpi=100, bbox_inches='tight')    
    plt.close(fig)

In [4]:
def close_image(image, kernel_size):
    '''Returns the image that is closed with a kernel.'''
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_size, kernel_size))
    closing = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    
    return closing

In [5]:
def extract_longest_contour(image, kernel_size):
    '''Returns the contour with the most points from a given image.'''
    
    # Close image
    closing = close_image(image, kernel_size)
    
    # Convert image to grayscale
    gray = cv2.cvtColor(closing, cv2.COLOR_BGR2GRAY)

    # Apply threshold to turn it into binary image
    ret, thresh = cv2.threshold(gray, 127, 255, 0)

    # Find contour
    # Change method for different number of points:
    # CHAIN_APPROX_NONE, CHAIN_APPROX_SIMPLE, CHAIN_APPROX_TC89_L1, CHAIN_APPROX_TC89_KCOS
    contours, _ = cv2.findContours(image=thresh,
                                   mode=cv2.RETR_TREE,
                                   method=cv2.CHAIN_APPROX_NONE)
    
    # Get longest contour from contours
    longest_contour = max(contours, key=len)
    
    # Remove single-dimensional entry from the shape of the array 
    longest_contour = np.squeeze(longest_contour)
    
    return longest_contour

In [6]:
def shorten_contour(contour, points):
    '''Returns a contour with a fixed number of points for a given contour.'''
    
    try:
        step = len(contour) / (len(contour) - points)
    except ZeroDivisionError as e:
        print ('len(contour) = {}, points = {}'.format(len(contour), points))
        raise ValueError from e
        
    short_contour = np.delete(contour, np.arange(0, contour.size, step), axis=0)
    
    return short_contour

In [7]:
def get_contour_from_annotation(annotation_path, kernel_size, points):
    '''Returns the image, the longest contour, and shortened contour 
       given a annotation and fixed number of points.'''
    
    image = cv2.imread(annotation_path)
    contour = extract_longest_contour(image, kernel_size)
    try:
        short_contour = shorten_contour(contour, points)
    except ValueError as e:
        print(annotation_path)
        raise ValueError from e
        
    return image, contour, short_contour

In [8]:
def create_contours_for_all_annotations(annotations_folders_path, 
                                        contours_folders_path, 
                                        kernel_size,
                                        contour_points):

    annotations_folders_list = os.listdir(annotations_folders_path)
    
    # Iterate through folders
    for i, folder in enumerate(annotations_folders_list):
        
        print('#{}: {}'.format(i, folder))
        
        # Create folder to save contours
        contours_folder = os.path.join(contours_folders_path, folder)
        if not os.path.exists(contours_folder):
            os.makedirs(contours_folder)
        
        # Get list of annotations
        annotations_list = os.listdir(os.path.join(annotations_folders_path, folder))
        if '.ipynb_checkpoints' in annotations_list:
            annotations_list.remove('.ipynb_checkpoints')
            
        # Iterate through annotations
        for j, annotation in enumerate(annotations_list):

            annotation_path = os.path.join(annotations_folders_path, folder, annotation)

            # If first mask, then extract contour_points
            if re.match(r"00000.png", annotation):
                try:
                    image, contour, short_contour = get_contour_from_annotation(
                        annotation_path, kernel_size, contour_points)
                except ValueError as e:
                    continue
            # If not, then contour_points*2
            else:
                try:
                    image, contour, short_contour = get_contour_from_annotation(
                        annotation_path, kernel_size, contour_points * 2)
                except ValueError as e:
                    continue

            # Save contour
            save_image_with_contour(image, short_contour, os.path.join(contours_folder, annotation))
            np.save(os.path.join(contours_folder, annotation), short_contour)

## Create Contours

In [10]:
ANNOTATIONS_480P_FOLDERS_PATH = "DAVIS_2016/DAVIS/Annotations/480p/"
CONTOURS_480P_FOLDERS_PATH = "DAVIS_2016/DAVIS/Contours/480p/"

ANNOTATIONS_1080P_FOLDERS_PATH = "DAVIS_2016/DAVIS/Annotations/1080p/"
CONTOURS_1080P_FOLDERS_PATH = "DAVIS_2016/DAVIS/Contours/1080p/"

KERNEL_SIZE = 25
CONTOUR_POINTS = 256

create_contours_for_all_annotations(ANNOTATIONS_480P_FOLDERS_PATH,
                                    CONTOURS_480P_FOLDERS_PATH,
                                    KERNEL_SIZE, 
                                    CONTOUR_POINTS)

#0: bus


/home/christoph/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
  # This is added back by InteractiveShellApp.init_path()
/home/christoph/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  # This is added back by InteractiveShellApp.init_path()


#1: scooter-black
#2: hike
#3: rollerblade
#4: hockey
#5: breakdance-flare
#6: motocross-jump
#7: dance-twirl
#8: dog
#9: train
#10: lucia
#11: mallard-fly
#12: libby
#13: boat
#14: camel
#15: parkour
#16: paragliding
#17: bear
#18: horsejump-high
#19: scooter-gray
#20: paragliding-launch
#21: soapbox
#22: swing
#23: soccerball
#24: kite-walk
#25: motorbike
#26: drift-straight
#27: dance-jump
#28: bmx-trees
#29: blackswan
#30: car-shadow
#31: rhino
#32: drift-turn
#33: tennis
#34: drift-chicane
#35: horsejump-low
#36: flamingo
#37: breakdance
#38: goat
#39: mallard-water
#40: motocross-bumps
#41: elephant
#42: stroller
#43: cows
#44: dog-agility
#45: car-turn
#46: car-roundabout
#47: kite-surf
#48: bmx-bumps
#49: surf


In [ ]:
# Do not use the following sequences
bad_sequences = ['bmx-trees', 'bus', 'cows', 'dog-agility', 'horsejump-high', 'horsejump-low', 
                 'kite-walk', 'lucia', 'libby', 'motorbike', 'paragliding', 'rhino', 'scooter-gray', 
                 'swing']